In [1]:
# Import all dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
# Read in data file
charity_df = pd.read_csv("charity_data.csv")
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop columns that we don't need for the model
charity_df = charity_df.drop(["EIN", "NAME"],1)

In [4]:
# Review application types to see how they can be categorized
application_types = charity_df.APPLICATION_TYPE.value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Use 500 as a cutoff for All Other types
# Create a variable for all application types under 500
ao_types = list(application_types[application_types < 500].index)

In [6]:
# Replace those in the dataframe
for type in ao_types:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(type,"Other")

In [7]:
# View new application types
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at classification column for bucketing
classification_types = charity_df.CLASSIFICATION.value_counts()
classification_types.head(25)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
Name: CLASSIFICATION, dtype: int64

In [14]:
# Use 1000 as cutoff for All Other classifications
# Create a variable for all classification types under 1000
ao_class = list(classification_types[classification_types < 1000].index)

In [19]:
# Replace those in the dataframe
for cls in ao_class:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(cls,"Other")

In [20]:
# View new classification types
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [21]:
# Create variable list for categorization
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [22]:
# Create instance of OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit and transform OneHotEncoder
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add variable names back into dataframe
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Merge new features and drop originals
charity_df = charity_df.merge(encode_df, left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# Data is now preprocessed
# Split data into features and targets
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

In [25]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
# Create instance of StandardScaler
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the deep neural network model
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

In [28]:
# Create first hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Create second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Create output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [29]:
# Check model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Put model together
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with 100 epochs
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 6s 252us/sample - loss: 0.5699 - accuracy: 0.7233
Epoch 2/100
25724/25724 [==============================] - 4s 153us/sample - loss: 0.5559 - accuracy: 0.7314
Epoch 3/100
25724/25724 [==============================] - 4s 159us/sample - loss: 0.5521 - accuracy: 0.7326
Epoch 4/100
25724/25724 [==============================] - 5s 177us/sample - loss: 0.5514 - accuracy: 0.7317
Epoch 5/100
25724/25724 [==============================] - 4s 166us/sample - loss: 0.5504 - accuracy: 0.7339
Epoch 6/100
25724/25724 [==============================] - 4s 144us/sample - loss: 0.5491 - accuracy: 0.7338
Epoch 7/100
25724/25724 [==============================] - 4s 144us/sample - loss: 0.5490 - accuracy: 0.7330
Epoch 8/100
25724/25724 [==============================] - 4s 148us/sample - loss: 0.5479 - accuracy: 0.7355
Epoch 9/100
25724/25724 [==============================] - 4s 145us/sample - loss: 0.5472 - accuracy: 0.7

25724/25724 [==============================] - 4s 146us/sample - loss: 0.5373 - accuracy: 0.7405
Epoch 76/100
25724/25724 [==============================] - 4s 159us/sample - loss: 0.5380 - accuracy: 0.7398
Epoch 77/100
25724/25724 [==============================] - 4s 147us/sample - loss: 0.5374 - accuracy: 0.7399
Epoch 78/100
25724/25724 [==============================] - 4s 140us/sample - loss: 0.5376 - accuracy: 0.7400
Epoch 79/100
25724/25724 [==============================] - 4s 155us/sample - loss: 0.5374 - accuracy: 0.7399
Epoch 80/100
25724/25724 [==============================] - 4s 152us/sample - loss: 0.5375 - accuracy: 0.7407
Epoch 81/100
25724/25724 [==============================] - 4s 149us/sample - loss: 0.5370 - accuracy: 0.7404
Epoch 82/100
25724/25724 [==============================] - 4s 150us/sample - loss: 0.5372 - accuracy: 0.7409
Epoch 83/100
25724/25724 [==============================] - 4s 148us/sample - loss: 0.5368 - accuracy: 0.7406
Epoch 84/100
25724/2572